In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
train_df = pd.read_csv ('Dataset/trainset.csv')
validation_df = pd.read_csv ('Dataset/validationset.csv')

data_train = train_df[['x_tr','future']].to_numpy()
data_validation = validation_df[['x_val','future']].to_numpy()

x_tr_string = data_train[:,0]
y_tr_string = data_train[:,1]
x_tr = []
y_tr = []
for i in x_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_tr = [int(j) for j in i.split()]
    x_tr.append(input_x_tr)

for i in y_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_tr = [int(j) for j in i.split()]
    y_tr.append(input_y_tr)
x_tr = np.array(x_tr)
y_tr = np.array(y_tr)
    
# print(x_tr.shape,y_tr.shape)

x_val_string = data_validation[:,0]
y_val_string = data_validation[:,1]
x_val = []
y_val = []
for i in x_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_val = [int(j) for j in i.split()]
    x_val.append(input_x_val)

for i in y_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_val = [int(j) for j in i.split()]
    y_val.append(input_y_val)
x_val = np.array(x_val)
y_val = np.array(y_val)

# print(x_val.shape,y_val.shape)

notes_ = notes_df.to_numpy()[:,1]
unique_notes_reverse = dict(enumerate(notes_.flatten(), 1))
unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}

In [3]:
from MusicDataset import *
batch_size = 8
train_set = MusicDataset(x_tr,y_tr)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [4]:
val_set = MusicDataset(x_val,y_val)
validationloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [5]:
from Models import Wavenet,bidirectionalLSTM
# Net = Wavenet.Wavenet(len(unique_notes),blocks=4,layers = 2)
Net = bidirectionalLSTM.Sequence(len(unique_notes),32)
Net.to(device)

Sequence(
  (embedding): Embedding(10, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear1): Linear(in_features=512, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Sequence(
  (embedding): Embedding(10, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear1): Linear(in_features=512, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


Sequence(
  (embedding): Embedding(182, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear1): Linear(in_features=512, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=182, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [7]:
import os
import time

baseDir = r'C:\Users\Dell\music_generation/Music-Gen-AI'
expDir = os.path.join(baseDir, 'trained_model_cache', time.strftime("%d_%m_%Y_%H_%M"))
lossDir = os.path.join(expDir, 'loss')
os.makedirs(expDir, exist_ok=True)
os.makedirs(lossDir, exist_ok=True)
print(baseDir, expDir)

C:\Users\Dell\music_generation/Music-Gen-AI C:\Users\Dell\music_generation/Music-Gen-AI\trained_model_cache\03_04_2024_16_47


In [8]:
loss_history = []
best_val_acc = 0
training_mode_list = ["Next_Predict_Mode","Future_Predict_Mode"]
training_mode = "Next_Predict_Mode"
# training_mode = "Future_Predict_Mode"
future_preds = 5

In [13]:
import timeit

''' TRAINING THE MODEL '''
correct_preds = 0
total_preds = 0
for epoch in range(20):
    running_loss = 0.0
    start = timeit.default_timer()
    for i, data in enumerate(trainloader, 0):
        input, label = data
        optimizer.zero_grad()
        label = label[:, 0].long()  # Ensure label type is Long
        if training_mode == training_mode_list[0]:
            output = Net(input.to(device))
            loss = criterion(output.to(device), label.to(device))
            loss.backward()
            optimizer.step()
            total_preds += input.shape[0]
            correct_preds += torch.sum(torch.argmax(output, 1) == label.to(device))

            running_loss += loss.item()
        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = np.array([], dtype=int)
            for k in range(future_preds):
                output = Net(input.to(device))
                cumm_output = torch.cat((cumm_output, output))
                cumm_label = np.concatenate((cumm_label, label[:, k]))
                next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
                input = input.cpu().detach().numpy()
                input = torch.from_numpy(np.array([np.append(j, next_preds[ind])
                                                    for ind, j in enumerate(input)])[:, 1:])
            loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if (i % 300 == 299):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            loss_history.append(running_loss / 300)
            running_loss = 0.0
    stop = timeit.default_timer()
    print(f"Training Time for Epoch :{epoch} \n {stop-start} seconds\n")
    train_acc = float(correct_preds) / float(total_preds) * 100
    trainreport = "Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Training Accuracy : {} \n ------------------------\n".format(
        correct_preds, total_preds, train_acc)
    print(trainreport)
    correct_preds = 0
    total_preds = 0
    val_loss = 0
    start = timeit.default_timer()
    for i, data in enumerate(validationloader, 0):
        input, label = data
        label = label[:, 0].long()  # Ensure label type is Long
        if training_mode == training_mode_list[0]:
            output = Net(input.to(device))
            loss = criterion(output.to(device), label.to(device))
            total_preds += input.shape[0]
            correct_preds += torch.sum(torch.argmax(output, 1) == label.to(device))
            val_loss += loss.item()

        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = np.array([], dtype=int)
            for k in range(future_preds):
                output = Net(input.to(device))
                cumm_output = torch.cat((cumm_output, output))
                cumm_label = np.concatenate((cumm_label, label[:, k]))
                next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
                input = input.cpu().detach().numpy()
                input = torch.from_numpy(np.array([np.append(j, next_preds[ind])
                                                    for ind, j in enumerate(input)])[:, 1:])
            loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))

            val_loss += loss.item()
    stop = timeit.default_timer()
    print(f"Validation Time for Epoch :{epoch} \n {stop-start} seconds\n")
    val_acc = float(correct_preds) / float(total_preds) * 100
    val_loss = float(val_loss) / float(total_preds)
    valreport = "Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n Val-Accuracy :{}----------------------------------\n".format(
        correct_preds, total_preds, val_loss, val_acc)
    print(valreport)

    if (val_acc > best_val_acc):
        print('Updating the model')
        best_val_acc = val_acc
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": Net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_acc": train_acc,
            "valid_acc": val_acc
        }
        torch.save(checkpoint, os.path.join(expDir, 'checkpoint1.tar'))
        torch.save(Net, os.path.join(expDir, 'model1.pth'))

    correct_preds = 0
    total_preds = 0

print('Finished Training')

[1,   300] loss: 4.926
[1,   600] loss: 4.818
[1,   900] loss: 4.749
[1,  1200] loss: 4.714
[1,  1500] loss: 4.691
[1,  1800] loss: 4.713
[1,  2100] loss: 4.694
[1,  2400] loss: 4.690
[1,  2700] loss: 4.682
[1,  3000] loss: 4.640
[1,  3300] loss: 4.648
[1,  3600] loss: 4.641
[1,  3900] loss: 4.593
[1,  4200] loss: 4.612
[1,  4500] loss: 4.622
[1,  4800] loss: 4.576
[1,  5100] loss: 4.624
[1,  5400] loss: 4.628
[1,  5700] loss: 4.561
[1,  6000] loss: 4.556
[1,  6300] loss: 4.587
[1,  6600] loss: 4.526
Training Time for Epoch :0 
 1236.878060100018 seconds

Training Accuracy : 
 correct predictions  : 1435 
 total predictions : 54873 
 Training Accuracy : 2.6151294808011225 
 ------------------------

Validation Time for Epoch :0 
 86.31152240000665 seconds

Validation Accuracy : 
 correct predictions  : 632 
 total predictions : 15678 
 Val-Loss : 0.5646740222162762 
 Val-Accuracy :4.031126419186121----------------------------------

Updating the model
[2,   300] loss: 4.473
[2,   600] 

[9,  6300] loss: 3.262
[9,  6600] loss: 3.175
Training Time for Epoch :8 
 554.5692437000107 seconds

Training Accuracy : 
 correct predictions  : 11472 
 total predictions : 54873 
 Training Accuracy : 20.90645672735225 
 ------------------------

Validation Time for Epoch :8 
 43.54307529999642 seconds

Validation Accuracy : 
 correct predictions  : 2867 
 total predictions : 15678 
 Val-Loss : 0.44361609737549773 
 Val-Accuracy :18.286771271845897----------------------------------

Updating the model
[10,   300] loss: 3.201
[10,   600] loss: 3.171
[10,   900] loss: 3.107
[10,  1200] loss: 3.140
[10,  1500] loss: 3.113
[10,  1800] loss: 3.137
[10,  2100] loss: 3.167
[10,  2400] loss: 3.189
[10,  2700] loss: 3.187
[10,  3000] loss: 3.132
[10,  3300] loss: 3.213
[10,  3600] loss: 3.147
[10,  3900] loss: 3.157
[10,  4200] loss: 3.165
[10,  4500] loss: 3.097
[10,  4800] loss: 3.057
[10,  5100] loss: 3.111
[10,  5400] loss: 3.163
[10,  5700] loss: 3.074
[10,  6000] loss: 3.130
[10,  6300]

[18,   300] loss: 2.846
[18,   600] loss: 2.780
[18,   900] loss: 2.781
[18,  1200] loss: 2.744
[18,  1500] loss: 2.714
[18,  1800] loss: 2.720
[18,  2100] loss: 2.801
[18,  2400] loss: 2.853
[18,  2700] loss: 2.809
[18,  3000] loss: 2.790
[18,  3300] loss: 2.871
[18,  3600] loss: 2.777
[18,  3900] loss: 2.786
[18,  4200] loss: 2.819
[18,  4500] loss: 2.782
[18,  4800] loss: 2.649
[18,  5100] loss: 2.782
[18,  5400] loss: 2.810
[18,  5700] loss: 2.734
[18,  6000] loss: 2.793
[18,  6300] loss: 2.821
[18,  6600] loss: 2.788
Training Time for Epoch :17 
 586.5325863999897 seconds

Training Accuracy : 
 correct predictions  : 16544 
 total predictions : 54873 
 Training Accuracy : 30.1496182093197 
 ------------------------

Validation Time for Epoch :17 
 48.897466800000984 seconds

Validation Accuracy : 
 correct predictions  : 3650 
 total predictions : 15678 
 Val-Loss : 0.43415637427043025 
 Val-Accuracy :23.28103074371731----------------------------------

Updating the model
[19,   3

In [1]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(loss_history)
plt.savefig(lossDir + '/loss_history.png')

NameError: name 'loss_history' is not defined

<Figure size 640x480 with 0 Axes>